# Week 10


## Gloria Moore

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install nltk
import nltk
from pathlib import Path
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
sentence = """Airplane Crashes has been decreasing from 1995 to 2020 in about 31% worldwide, this is an indicative that flying is becoming safer over the years"""

In [3]:
import nltk
nltk.download('punkt')
import re
from nltk import sent_tokenize, word_tokenize, pos_tag_sents

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def tokenize(sentence):
    #removing punctuation
    res = re.sub(r'[^\w\s]', '', sentence)
    #removing numbers from a string
    res = re.sub(r"\d", "", res)
    # tokenize the sentence
    tokens = nltk.word_tokenize(res)
       
    return tokens

In [5]:
tokens = tokenize(sentence)

In [6]:
tokens

['Airplane',
 'Crashes',
 'has',
 'been',
 'decreasing',
 'from',
 'to',
 'in',
 'about',
 'worldwide',
 'this',
 'is',
 'an',
 'indicative',
 'that',
 'flying',
 'is',
 'becoming',
 'safer',
 'over',
 'the',
 'years']

#### Assignment 10.1.b. Implement an `ngram` function that splits tokens into N-grams. 

In [14]:
def ngram(tokens, n):
    ngrams = []
    # Create ngrams
    for i in range(len(tokens)-n+1):
        ngrams.append(tokens[i:i+n])
        
    
    return [" ".join(a) for a in ngrams]

In [15]:
ngram(tokens, 2)

['Airplane Crashes',
 'Crashes has',
 'has been',
 'been decreasing',
 'decreasing from',
 'from to',
 'to in',
 'in about',
 'about worldwide',
 'worldwide this',
 'this is',
 'is an',
 'an indicative',
 'indicative that',
 'that flying',
 'flying is',
 'is becoming',
 'becoming safer',
 'safer over',
 'over the',
 'the years']

### Assignment 10.1.c. Implement an one_hot_encode function to create a vector from a numerical vector from a list of tokens.

In [16]:
#Cheeking other book
def vectorize(tokens):
    return {token: True for token in tokens}
vectors = map(vectorize, tokens)

In [17]:
#from our tex Book p.182

def one_hot_encode(tokens, num_words):
    token_index = {}
    for sample in samples:
        for word in sample.split():
            if word not in token_index:
                token_index[word] = len(token_index) + 1
                results = np.zeros(shape = (len(samples), num_words, max(token_index.values()) + 1))
    for i, sample in enumerate(samples):
        for j, word in list(enumerate(sample.split()))[ :num_words]:
            index = token_index.get(word)
            results[i, j, index] = 1
    return results


num_words = 10
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
one_hot_encode(tokens, num_words)

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0

##### 10.2. 

Using listings 6.16, 6.17, and 6.18 in Deep Learning with Python as a guide, train a sequential model with embeddings on the IMDB data found in data/external/imdb/. Produce the model performance metrics and training and validation accuracy curves within the Jupyter notebook.

In [19]:
#p 193

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [26]:
current_dir = Path(os.getcwd()).absolute()
imdb_dir = Path('/home/jovyan/dsc650/data/external/imdb/aclImdb/')
test_dir = os.path.join(imdb_dir, 'test')
train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [27]:
maxlen = 100
training_samples = 200
validation_samples = 10000
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Found 88582 unique tokens.
Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [28]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history = model.fit(x_train, y_train,
epochs=10,
batch_size=32,
validation_data=(x_val, y_val))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
7/7 [==============================] - 1s 133ms/step - loss: 0.6971 - acc: 0.5052 - val_loss: 0.6918 - val_acc: 0.5228
Epoch 2/10
7/7 [==============================] - 1s 111ms/step - loss: 0.5246 - acc: 1.0000 - val_loss: 0.6928 

In [30]:
#6.17. p. 194
test_dir = os.path.join(imdb_dir, 'test')
labels = []
texts = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in sorted(os.listdir(dir_name)):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)
sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

In [32]:
#6.14 

model.compile(optimizer='rmsprop',
loss='binary_crossentropy',metrics=['acc'])
history = model.fit(x_train, y_train,epochs=10,
batch_size=32,
validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')

Epoch 1/10
7/7 [==============================] - 1s 128ms/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.7910 - val_acc: 0.5382
Epoch 2/10
7/7 [==============================] - 1s 108ms/step - loss: 1.2721e-04 - acc: 1.0000 - val_loss: 0.7886 - val_acc: 0.5370
Epoch 3/10
7/7 [==============================] - 1s 105ms/step - loss: 5.8448e-05 - acc: 1.0000 - val_loss: 0.7979 - val_acc: 0.5359
Epoch 4/10
7/7 [==============================] - 1s 107ms/step - loss: 3.5699e-05 - acc: 1.0000 - val_loss: 0.8051 - val_acc: 0.5356
Epoch 5/10
7/7 [==============================] - 1s 107ms/step - loss: 2.2800e-05 - acc: 1.0000 - val_loss: 0.8139 - val_acc: 0.5351
Epoch 6/10
7/7 [==============================] - 1s 108ms/step - loss: 1.5773e-05 - acc: 1.0000 - val_loss: 0.8209 - val_acc: 0.5365
Epoch 7/10
7/7 [==============================] - 1s 108ms/step - loss: 1.0921e-05 - acc: 1.0000 - val_loss: 0.8279 - val_acc: 0.5357
Epoch 8/10
7/7 [==============================] - 1s 107ms/step - 

In [33]:
#6.18 p. 195


model.load_weights('pre_trained_glove_model.h5')
model.evaluate(x_test, y_test)

782/782 [==============================] - 2s 2ms/step - loss: 0.8453 - acc: 0.5405


[0.8452569246292114, 0.5404800176620483]

#### 10.3

Using listing 6.27 in Deep Learning with Python as a guide, fit the same data with an LSTM layer. Produce the model performance metrics and training and validation accuracy curves within the Jupyter notebook.

In [34]:
# 6.27. p205

from keras.layers import LSTM



In [36]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.layers import Dense
from keras.layers import SimpleRNN
max_features = 10000
maxlen = 500
batch_size = 32
print('Loading data...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(
num_words=max_features)
print(len(input_train), 'train sequences')
print(len(input_test), 'test sequences')
print('Pad sequences (samples x time)')
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape)

Loading data...


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do t

25000 train sequences
25000 test sequences
Pad sequences (samples x time)
input_train shape: (25000, 500)
input_test shape: (25000, 500)


In [37]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])
history = model.fit(input_train, y_train,epochs=10,batch_size=128,validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 69s 425ms/step - loss: 0.5822 - acc: 0.6904 - val_loss: 0.3993 - val_acc: 0.8270
Epoch 2/10
157/157 [==============================] - 62s 396ms/step - loss: 0.2876 - acc: 0.8888 - val_loss: 0.2980 - val_acc: 0.8886
Epoch 3/10
157/157 [==============================] - 63s 399ms/step - loss: 0.2333 - acc: 0.9144 - val_loss: 0.3478 - val_acc: 0.8630
Epoch 4/10
157/157 [==============================] - 62s 393ms/step - loss: 0.1974 - acc: 0.9300 - val_loss: 0.3600 - val_acc: 0.8746
Epoch 5/10
157/157 [==============================] - 63s 401ms/step - loss: 0.1740 - acc: 0.9383 - val_loss: 0.3181 - val_acc: 0.8794
Epoch 6/10
157/157 [==============================] - 62s 393ms/step - loss: 0.1530 - acc: 0.9497 - val_loss: 0.9120 - val_acc: 0.7838
Epoch 7/10
157/157 [==============================] - 62s 396ms/step - loss: 0.1514 - acc: 0.9507 - val_loss: 0.4821 - val_acc: 0.8134
Epoch 8/10
157/157 [==============================] - 6

#### 10.4


Using listing 6.46 in Deep Learning with Python as a guide, fit the same data with a simple 1D convnet. Produce the model performance metrics and training and validation accuracy curves within the Jupyter notebook

In [38]:
from keras.datasets import imdb
from keras.preprocessing import sequence
max_features = 10000
max_len = 500
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 500)
x_test shape: (25000, 500)


In [39]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer=RMSprop(lr=1e-4),loss='binary_crossentropy',metrics=['acc'])
history = model.fit(x_train, y_train,
epochs=10,
batch_size=128,
validation_split=0.2)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 128)          1280000   
_________________________________________________________________
conv1d (Conv1D)              (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 98, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 1,315,937
Trainable params: 1,315,937
Non-trainable params: 0
____________________________________________